In [ ]:
import requests
from gnews import GNews
import json
import urllib.request
import sys
sys.path.append("../database")
import db_functions as db
import pandas as pd
import datetime

key = '9c91bc6d7cca946fffc04ee94bc2e24c'

In [ ]:
google_news = GNews(language='en', 
                    country='US', 
                    period='1d', 
                    max_results=10, 
                    # exclude_websites=['yahoo.com', 'cnn.com'],
                    )

In [ ]:
# https://docs.python.org/3/library/urllib.request.html#module-urllib.request
# This library will be used to fetch the API.

class Api_GNews():

    def __init__(self,api_key = '9c91bc6d7cca946fffc04ee94bc2e24c'):
        self.api_key = api_key
        self.db = db_functions.Database()
        print(f'Api_Gnews object and Database object were initialized')


    # def fetch_articles(self, keyword='Tesla'):
    #     url = f"https://gnews.io/api/v4/search?q={keyword}&lang=en&country=us&max=10&in=title&apikey={self.api_key}"
    #     with urllib.request.urlopen(url) as response:
    #         status_code = response.getcode()
    #         if status_code == 200:
    #             fetched_data = json.loads(response.read().decode("utf-8"))
    #             articles = fetched_data["articles"]
    #             print("Data fetched successfully!")
    #             return articles
    #         else:
    #             error_message = fetched_data.get("message")
    #             print(f"Error fetching data. Status code: {status_code}. Error message is {error_message}")


    def fetch_articles(self, keyword='Tesla'):
        
        from_time = int((datetime.now() - timedelta(hours=1)).timestamp())
        url = f"https://gnews.io/api/v4/search?q={keyword}&lang=en&mindate={from_time}&country=*"
        url += f"&max=10&in=title&apikey={self.api_key}"
        
        with urllib.request.urlopen(url) as response:
            status_code = response.getcode()
            if status_code == 200:
                fetched_data = json.loads(response.read().decode("utf-8"))
                articles = fetched_data["articles"]
                print("Data fetched successfully!")
                return articles
            else:
                error_message = fetched_data.get("message")
                print(f"Error fetching data. Status code: {status_code}. Error message is {error_message}")


    def upload_data(self, articles):
        for i in range(len(articles)):
            article = articles[i]
            data = {
                    "title": article["title"],
                    "description": article["description"],
                    "content": article["content"],
                    "url": article["url"],
                    "image": article["image"],
                    "publishedAt": article["publishedAt"],
                    "source": article["source"]["name"]
                    }
            self.db.insert_into_table('news',data)

In [ ]:
api = Api_GNews()

In [ ]:
articles = api.fetch_articles('Tesla')

In [ ]:
articles[0].keys()

In [ ]:
api.db.create_table(
    table_name = 'news',
    title = 'VARCHAR(255)',
    description = 'TEXT',
    content = 'TEXT',
    url = 'VARCHAR(255)',
    image = 'VARCHAR(255)',
    publishedAt = 'VARCHAR(255)',
    source = 'VARCHAR(255)'
)

In [ ]:
api.db.get_table_list()

In [ ]:
api.upload_data(articles)

In [ ]:
api.db.back_up_the_database()

In [ ]:
df_news = api.db.create_dataframe_from_table('news')

In [ ]:
df_news

In [ ]:
api.db.remove_duplicates('news', 'content')

In [ ]:
api.db.back_up_the_database()

In [ ]:
api.db.get_table_list()